In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import animation
from scipy.integrate import solve_ivp
from tools import * 
import sys
import os


def displayH(H):
    s =  Function('s')
    c =  Function('c')
    display(H.replace( sin,s).replace( cos,c))

def skew(a):
    return Matrix([
        [0, -a[2], a[1]],
        [a[2], 0, -a[0]],
        [-a[1], a[0], 0]])

def unskew(a):
    return Matrix([
        [a[2,1]],
        [a[0,2]],
        [a[1,0]]])

def rotx(t):
    return Matrix([
        [1, 0, 0],
        [0, cos(t), -sin(t)],
        [0, sin(t), cos(t)]])

def roty(t):
    return Matrix([
        [cos(t), 0, sin(t)],
        [0, 1, 0],
        [-sin(t), 0, cos(t)]])

def rotz(t):
    return Matrix([
        [cos(t), -sin(t), 0],
        [sin(t), cos(t), 0],
        [0, 0, 1]])

In [2]:

#Define variables
t = symbols('t')

#for rear footpad (body 2)
x_2 = Function('x_2')(t)
y_2 = Function('y_2')(t)
z_2 = Function('z_2')(t)

alpha_2 = Function('alpha_2')(t)
beta_2 = Function('beta_2')(t)
gamma_2 = Function('gamma_2')(t)

#twist forward deck to rear deck
theta_52 = Function('theta_52')(t)


#relative angle caster to deck
theta_32 = Function('theta_32')(t)
theta_65 = Function('theta_65')(t)


#fixed caster angle
theta_c = symbols('theta_c')

#rotation of wheel
theta_43 = Function('theta_43')(t)
theta_76 = Function('theta_76')(t)

#non-rotating wheel angle
xi_R = Function('xi_R')(t)
xi_F = Function('xi_F')(t)


q = Matrix([x_2,y_2,z_2,alpha_2,beta_2,gamma_2,theta_52,theta_32,theta_65,theta_43,theta_76,xi_R,xi_F])
qdot = diff(q,t)


In [3]:
#ROtation matrices
#rotation from inertial to rear footpad
#from 1 to 2
R12 = rotx(alpha_2)*roty(beta_2)*rotz(gamma_2)

#rear caster
#from 2 to 3
R23 = roty(-theta_c)*rotz(theta_32)

R13 = R12*R23  #2's cancel

#rear wheel
R34 = rotz(theta_43)
R14 = R13*R34 

#front footpad rotation
R25 = rotx(theta_52)
R15 = R12*R25 #2's cancel

#front caster
R56 = roty(-theta_c)*rotz(theta_65)
R16 = R15*R56 

R26 = R25*R56 

#front wheel
R67 = rotz(theta_76)
R17 = R16*R67

R27 = R26*R67

#non-rotating wheel frames
Rxi_R = roty(xi_R)
Rxi_F = roty(xi_F)


In [4]:
#positions of various things

l_z, l_cx, l_cz, l_x1, l_x2, l_x3 = symbols('l_z l_cx l_cz l_x1 l_x2 l_x3')
#lz = vertical difference between footpad and caster
#lcx = x distance between caster com and wheel com
#lcz = z ""
#lx1 = distance between two footpad's coms
#lx2 = distance between G2 and G3
#lx3 = distance between G5 and G6'

#location of com of rear footpad in inertial frame (absolute position)
r_G2_1 = Matrix([
    [x_2],
    [y_2],
    [z_2]])

v_G2_1 = diff(r_G2_1, t)
# print(v_G2_1)

#position of rear caster (body3) in 2-frame
r_G3_2 = Matrix([
    [-l_x2],
    [0],
    [-l_z]])

#position of rear caster in inertial frame
r_G3_1 =  r_G2_1 + R12*r_G3_2 

#position rear wheel in caster frame
r_G4_3 = Matrix([
    [-l_cx],
    [0],
    [-l_cz]])

#position of rear wheel in 2 frame
r_G4_2 = R23*r_G4_3

#position of rear wheel in inertial frame
r_G4_1 = r_G3_1 + R13*r_G4_3

#position of front footpad(5) in rear(2)frame
r_G5_2 = Matrix([
    [l_x1],
    [0],
    [0]])

#front footpad(5) in inertial frame
r_G5_1 =  r_G2_1 + R12*r_G5_2 

#front caster(6) in front footpad 5-frame
r_G6_5 = Matrix([
    [l_x3],
    [0],
    [-l_z]])

#front caster in inertial
r_G6_2 = R25*r_G6_5
r_G6_1 = r_G5_1 + R15*r_G6_5 

#front wheel in caster frame
r_G7_6 = Matrix([
    [-l_cx],
    [0],
    [-l_cz]])

#front wheel in inertial
r_G7_1 = r_G6_1 + R16*r_G7_6
r_G7_2 = R26*r_G7_6
 
#location of wheel contact with ground
R = symbols('R')
wheel_radius = Matrix([
    [R],
    [0],
    [0]])

#position of contact point on rear and front wheels   
r_R = r_G4_1 + R14*Rxi_R*wheel_radius
r_F = r_G7_1 + R17*Rxi_F*wheel_radius

#tangent vectors of wheels at R and F
t_R = R14* diff(Rxi_R*wheel_radius,xi_R)
t_F = R17 *diff(Rxi_F*wheel_radius,xi_F)


In [5]:
#constraints on the system
#holonomic constraints on the system Ch(q) = [CH1,CH2,CH3,CH4].T
CH1 = r_R[2]
CH2 = r_F[2]
normVect = Matrix([0,0,1]) #normal vector to ground in 1 frame
CH3 = t_R.dot(normVect)
CH4 = t_F.dot(normVect)

CH = Matrix([CH1,CH2,CH3,CH4]) #holonomic constraints  CH = 0

#non holonomic constriants on the system Cnh(q) = [CNH1,CNH2,CNH3,CNH4].T
#velocity vectors of the contact points F and R
v_R = r_R.diff(t) #need rotation matrix herer?
v_F = r_F.diff(t)

CNH1= v_R[0]
CNH2 = v_R[1]
CNH3 = v_F[0]
CNH4 = v_F[1]

CNH = Matrix([CNH1,CNH2,CNH3,CNH4]) #non holonomic constraints CNH=0
print(shape(CNH))

#finding the holonomic velocity of lagranges equation?? not sure what its called 
q = Matrix([x_2, 
            y_2, 
            z_2, 
            alpha_2, 
            beta_2, 
            gamma_2, 
            theta_52, 
            theta_32, 
            theta_65, 
            theta_43, 
            theta_76, 
            xi_R, 
            xi_F])

CH_dot = CH.diff(t) 
CH_q = CH_dot.jacobian(qdot) #CH_dt * qdot = 0
print(shape(CH_q))


B = CNH.jacobian(qdot) 
print(shape(B))
D = Matrix([[CH_q],
            [B]])  #D(q)qdot = 0
print(shape(D))

(4, 1)
(4, 13)
(4, 13)
(8, 13)


In [6]:
#setting up mass matrices
mp = 1.14 #kg mass of decks
mh=  70 #kg mass of human

m2 = mp+mh/2
m5 = m2
m3 = 0.25 #mass of casters
m6 = m3
m4 = 0.11 #mass of wheels
m7 = m4 
